# MODEL EVALUATION

In [ ]:
import os 
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\End-to-End-Machine-Leaning-Project-With-Mlow\\research"

In [1]:
os.chdir('../')
%pwd

NameError: name 'os' is not defined

## DagsHub setup

1. Open repo in dagshub
2. Open experiment

`MLFLOW_TRACKING_URI=https://dagshub.com/Zaheer-10/End-to-End-Machine-Leaning-Project-With-Mlow.mlflow \
MLFLOW_TRACKING_USERNAME=Zaheer-10 \
MLFLOW_TRACKING_PASSWORD=c73afb9df2eb3858963a808071f72dce3e66f85a \
python script.py`

3. RUN this to export as env variables

```bash 

export MLFLOW_TRACKING_URI=https://dagshub.com/Zaheer-10/End-to-End-Machine-Leaning-Project-With-Mlow.mlflow

export MLFLOW_TRACKING_USERNAME=Zaheer-10

export MLFLOW_TRACKING_PASSWORD=c73afb9df2eb3858963a808071f72dce3e66f85a


```
`NOTE : RUN THESE CMD IN GITBASH`

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/Zaheer-10/End-to-End-Machine-Leaning-Project-With-Mlow.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Zaheer-10'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'c73afb9df2eb3858963a808071f72dce3e66f85a'

In [ ]:
#1 update the config.yaml

## Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

# Config Manager

In [ ]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, parmas_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(parmas_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/Zaheer-10/End-to-End-Machine-Leaning-Project-With-Mlow.mlflow'
        )
        
        return model_evaluation_config

## Components



In [ ]:
import os
import joblib
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error , r2_score

In [ ]:
from mlproject.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self , actual , pred):
        rmse= np.sqrt(mean_squared_error(actual , pred))
        mae = mean_squared_error(actual , pred)
        r2 = r2_score(actual , pred)
        return rmse,mae,r2    
    
    # Mlflow tracking
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop([self.config.target_column] , axis=1)
        Y_test = test_data[[self.config.target_column]] 
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)
            
            (rmse , mae , r2_score) = self.eval_metrics(Y_test , predicted_qualities)
            
            #SAVING METRIC AS  LOCAL
            
            scores = {'rmse' : rmse , "mae" : mae , 'r2':r2_score}
            save_json(path=Path(self.config.metric_file_name) , data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric('rmse' , rmse)
            mlflow.log_metric('R2' , r2_score)
            mlflow.log_metric('mae' , mae)
            
            # Model registry doesn't work with file store
            if tracking_url_type_score != 'file' : 
                # register the model
                
                # mlflow.sklearn.log_model(model, 'model' , registered_model_name='ElasticNetModel')
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.load_model(model , 'model')

## Pipeline

In [ ]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e :
    raise e

[2023-11-25 21:03:35,198] : INFO : common :  yaml file: config\config.yaml loaded successfully]
[2023-11-25 21:03:35,220] : INFO : common :  yaml file: params.yaml loaded successfully]
[2023-11-25 21:03:35,226] : INFO : common :  yaml file: schema.yaml loaded successfully]
[2023-11-25 21:03:35,235] : INFO : common :  created directory at: artifacts]
[2023-11-25 21:03:35,238] : INFO : common :  created directory at: artifacts/model_evaluation]
[2023-11-25 21:03:36,384] : INFO : common :  json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2023/11/25 21:03:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 5
Created version '5' of model 'ElasticNetModel'.
